## 图像分割

- 图像分割任务，即通过给出图像中每个像素点的标签，将图像分割成若干带类别标签的区块，可以看作对每个像素进行分类。
- PaddleSeg覆盖了DeepLabv3+、U-Net、PSPNet、HRNet和Fast-SCNN等20+主流分割模型，并提供了多个损失函数和多种数据增强方法等高级功能。
- 在图像分割领域中，评估模型质量主要是通过三个指标进行判断，准确率（acc）、平均交并比（Mean Intersection over Union，简称mIoU）、Kappa系数。
    - 准确率：指类别预测正确的像素占总像素的比例，准确率越高模型质量越好。
    - 平均交并比：对每个类别数据集单独进行推理计算，计算出的预测区域和实际区域交集除以预测区域和实际区域的并集，然后将所有类别得到的结果取平均。
    - Kappa系数：一个用于一致性检验的指标，可以用于衡量分类的效果。kappa系数的计算是基于混淆矩阵的，取值为-1到1之间，通常大于0。

## 文本分类

- 文本分类是指使用计算机将文本数据进行自动化归类的任务，是自然语言处理（NLP）中的一项重要任务。
- ERNIE是一个预训练模型，使用三种级别的Knowledge Masking帮助模型学习语言知识，在多项任务上超越了BERT。在模型结构方面，它采用了Transformer的Encoder部分作为模型主干进行训练。
    - Seq2Seq模型: sequence to sequence模型是一类End-to-End的算法框架，也就是从序列到序列的转换模型框架，应用在机器翻译，自动应答等场景。
    - Attention机制（注意力）：注意力机制可以利用人类的认知机制直观解释。例如，我们的视觉系统倾向于关注图像中辅助判断的部分信息，并忽略掉不相关的信息。同样，在自然语言处理的问题中，输入的某些部分可能会比其他部分对决策更有帮助。
    - Transfomer模型：很多NLP的语义学习问题涉及到大量的训练数据，而RNN类的模型内部存在计算依赖，无法高效的并行化训练。使用Self-attenion的方法，将RNN变成每个输入与其他输入部分计算匹配度来决定注意力权重的方式，使得模型引入了Attention机制的同时也具备了并行化计算的能力。以这种Self-attention结构为核心，设计Encoder-Decoder的结构形成Transformer模型。

## 使用开发套件ERNIE实现对新闻标题的分类

### 数据集介绍

THUCNews是根据新浪新闻RSS订阅频道2005~2011年间的历史数据筛选过滤生成，均为UTF-8纯文本格式。在原始新浪新闻分类体系的基础上，重新整合划分出14个候选分类类别：财经、彩票、房产、股票、家居、教育、科技、社会、时尚、时政、体育、星座、游戏、娱乐。

本案例使用的数据集是从THUCNews新闻数据中根据新闻类别按照一定的比例提取了新闻标题，其中训练集数据约27.1w，测试集约6.7w条，另有一份记录标签的词表`lable_dict.txt`。

In [3]:
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddle.io import Dataset

import paddlenlp
from paddlenlp.datasets import MapDataset
from paddlenlp.data import Stack, Tuple, Pad
from paddlenlp.transformers import LinearDecayWithWarmup

import numpy as np
from functools import partial

In [6]:
# 构建数据集类
class NewsDataset(Dataset):
    def __init__(self, data_path, label_path):
        # 加载标签词典
        self.label2id = self._load_label_dict(label_path)
        
        self.label_list = list(self.label2id.keys())
        
        # 加载数据集
        self.data = self._load_data(data_path)
        
    def _load_data(self, data_path):
        dataset = []
        
        with open(data_path, 'r', encoding='utf-8') as f:
            for line in f.readlines():
                label, text = line.strip().split('\t', maxsplit=1)
                example = {'text': text, 'label': self.label2id[label]}
                dataset.append(example)
                
        return dataset
                
    def _load_label_dict(self, label_path):
        with open(label_path, 'r', encoding='utf-8') as f:
            lines = [line.strip().split() for line in f.readlines()]
            lines = [(line[0], int(line[1])) for line in lines]
            label_dict = dict(lines)
            
        return label_dict 
    
    def __getitem__(self, idx):
        return self.data[idx]
    
    def __len__(self):
        return len(self.data)

In [14]:
data_path = '../datasets/THUCNews/train.txt'
label_path = '../datasets/THUCNews/label_dict.txt'

news_dataset = NewsDataset(data_path, label_path)

print('训练集长度：', news_dataset.__len__())
print('训练集格式：', news_dataset.__getitem__(0))

print('标签字典：', news_dataset.label2id)
print('标签列表：', news_dataset.label_list)

训练集长度： 271167
训练集格式： {'text': '爱情测试，你的爱情年老时是啥样', 'label': 0}
标签字典： {'星座': 0, '科技': 1, '房产': 2, '股票': 3, '彩票': 4, '时尚': 5, '教育': 6, '体育': 7, '娱乐': 8, '家居': 9, '时政': 10, '社会': 11, '财经': 12, '游戏': 13}
标签列表： ['星座', '科技', '房产', '股票', '彩票', '时尚', '教育', '体育', '娱乐', '家居', '时政', '社会', '财经', '游戏']


In [15]:
# 构建数据加载器
def create_dataloader(dataset, mode='train', batch_size=1, batchify_fn=None, trans_fn=None):
    # 使用该函数将数据处理成模型输入需要的格式
    if trans_fn:
        dataset = dataset.map(trans_fn)
        
    shuffle = True if mode == 'train' else False
    
    if mode == 'train':
        batch_sampler = paddle.io.DistributeBatchSampler(dataset, batch_size=batch_size, shuffle=shuffle)
    else:
        batch_sampler = paddle.io.BatchSampler(dataset, batch_size=batch_size, shuffle=shuffle)
        
    return paddle.io.DataLoader(
        dataset=dataset,
        batch_sampler=batch_sampler,
        collate_fn=batchify_fn,
        return_list=True
    )

In [16]:
MODEL_NAME = 'ernie-1.0'
# 在转换数据格式的时候，调用paddleNLP封装好的tokenizer
tokenizer = paddlenlp.transformers.ErnieTokenizer.from_pretrained(MODEL_NAME)

[2023-01-09 16:48:59,232] [    INFO] - Downloading https://bj.bcebos.com/paddlenlp/models/transformers/ernie/vocab.txt and saved to /Users/neowong/.paddlenlp/models/ernie-1.0
[2023-01-09 16:48:59,235] [    INFO] - Downloading vocab.txt from https://bj.bcebos.com/paddlenlp/models/transformers/ernie/vocab.txt
100%|██████████████████████████████████████| 89.5k/89.5k [00:00<00:00, 19.2MB/s]
[2023-01-09 16:48:59,674] [    INFO] - tokenizer config file saved in /Users/neowong/.paddlenlp/models/ernie-1.0/tokenizer_config.json
[2023-01-09 16:48:59,675] [    INFO] - Special tokens file saved in /Users/neowong/.paddlenlp/models/ernie-1.0/special_tokens_map.json


In [17]:
# 规整化mini-batch数据（长短不一）为模型期望的样式
batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),       # input
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # segment
    Stack(dtype='int64')  # label
): [data for data in fn(samples)]